In [35]:
#  imports
import os
import sys
import time
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mne
from config import cfg

In [13]:
data_path = "/Users/rufaelmarew/Documents/tau/finger_pose_estimation/dataset/FPE/S1/p3/fpe_pos3_001_S1_rep0_BT.edf"
leap_path = "/Users/rufaelmarew/Documents/tau/finger_pose_estimation/dataset/FPE/S1/p3/fpe_pos3_001_S1_rep0_BT.csv"

In [14]:
def read_emg(path, start_time=None, end_time=None, fs: int=250):

    raw = mne.io.read_raw_edf(path, preload=True)

    # get header
    header = raw.info

    if start_time is None:
        start_time = header['meas_date']
        # convert to pd.datetime from datetime.datetime
        start_time = pd.to_datetime(start_time).tz_localize(None)
        # remove 2 hours
        start_time = start_time - pd.to_timedelta(2, unit='h')
        print(start_time)
    
    #  get annotations
    annotations = raw.annotations
    annotations.onset = start_time + pd.to_timedelta(annotations.onset, unit='s')
    
    # get annotations as df
    to_append = []
    for ind, (i,j) in enumerate(zip(annotations.onset, annotations.description)):
        if 'start' in j:
            if 'end_' in annotations.description[ind+1]:
                new_j = j.replace('start', '').strip('_')
                #  add 1 sec ofset to onset and append
                offset = pd.to_timedelta(1, unit='s')
                to_append.append([annotations.onset[ind]+offset, annotations.onset[ind+1]+offset, new_j])
        
    ann_df = pd.DataFrame(to_append, columns=['start_time', 'end_time', 'gesture'])
    #  if duration is greater than 10 sec, drop
    ann_df = ann_df[ann_df['end_time'] - ann_df['start_time'] < pd.to_timedelta(10, unit='s')]


    emg_df = raw.to_data_frame()
    emg_df['time'] = pd.to_datetime(emg_df['time'], unit='s', origin=start_time)
    emg_df.set_index('time', inplace=True)

    # start data from first annotation
    start_time = ann_df['start_time'].iloc[0]
    emg_df = emg_df[start_time:]
    
    #  resample emg data to fs Hz
    emg_df = emg_df.resample(f'{int(1000/fs)}ms', origin='start').mean()

    return emg_df, ann_df, header

emg_data, ann, header = read_emg(path = data_path)

Extracting EDF parameters from /Users/rufaelmarew/Documents/tau/finger_pose_estimation/dataset/FPE/S1/p3/fpe_pos3_001_S1_rep0_BT.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 135499  =      0.000 ...   541.996 secs...
2024-01-02 11:17:25


In [15]:
def read_leap(path, fs=125):

    leap_df = pd.read_csv(path, index_col=False)

    # drop null and duplicates
    leap_df.dropna(inplace=True)
    leap_df.drop_duplicates(inplace=True, subset=['time'])

    leap_df['time'] = pd.to_datetime(leap_df['time'])
    leap_df['time'] = leap_df['time'].dt.tz_localize(None)
    leap_df = leap_df.set_index('time')

    # calculate relative position
    for i in leap_df.columns:
        if 'position_x' in i:
            leap_df[i] = leap_df[i] - leap_df['palm_x']
        elif 'position_y' in i:
            leap_df[i] = leap_df[i] - leap_df['palm_y']
        elif 'position_z' in i:
            leap_df[i] = leap_df[i] - leap_df['palm_z']
        else:
            continue
    
    # leap_df = leap_df.resample(f'{int(1000/fs)}ms', origin='start').ffill()

    # valid_columns = build_leap_columns()
    # leap_df = leap_df[valid_columns]

    return leap_df, None, None

leap_data, _, _ = read_leap(leap_path)

In [16]:
start_time = max(min(leap_data.index), min(emg_data.index))
end_time = min(max(leap_data.index), max(emg_data.index))

leap_data = leap_data.loc[start_time:end_time]
emg_data = emg_data.loc[start_time:end_time]

In [17]:
def create_windowed_dataset(df, w, s, fs):
    # Convert window size and stride from seconds to number of rows
    w_rows = int(w * fs)
    s_rows = int(s * fs)
    print(w_rows, s_rows)
    print(df.shape)
    data = []
    times = []
    for i in range(0, len(df) - w_rows, s_rows):
        window = df.iloc[i:i+w_rows]
        data.append(window.values)
        times.append(window.index[-1])

    data = np.array(data)
    times = np.array(times)

    # Reshape data to (N-w)/(S)*W*C
    data = data.reshape((-1, w_rows , df.shape[1]))

    return data, times

data, times = create_windowed_dataset(emg_data, 1, 0.1, fs=250)
data.shape, times.shape

250 25
(125809, 16)


((5023, 250, 16), (5023,))

In [18]:
# get gesture if time is between start and end time
def get_gesture(time, ann_df):
    gesture_df = ann_df[(ann_df['start_time'] <= time) & (ann_df['end_time'] >= time)]
    if not gesture_df.empty:
        return gesture_df['gesture'].iloc[0]
    return 'rest'
t = get_gesture(times[0], ann)
print(t)

threefingers


In [20]:
# Get the indices of the leap data that are closest to the times array
def find_closest(leap_data, times, annotations):
    index = []
    gestures = []   
    for i in times:
        #  find the time indeex closest to i
        index.append(leap_data.index.asof(i))
        gestures.append(get_gesture(i,annotations))
        #  find the gesture closest to i
    leap_closest = leap_data.loc[index]
    
    return leap_closest.to_numpy(), gestures

index, gestures = find_closest(leap_data, times, ann)

In [22]:
data.shape, index.shape, len(gestures)

((5023, 250, 16), (5023, 147), 5023)

In [26]:
# value couts for each gesture
def get_gesture_counts(gestures):
    return pd.Series(gestures).value_counts()
get_gesture_counts(gestures)

rest            2527
twofingers       253
abduction        252
kaf              252
tet              251
het              251
fist             250
bet              250
gimel            248
nun              248
threefingers     241
Name: count, dtype: int64

In [27]:
import glob
def read_dirs(data_path):
    if not os.path.isdir(data_path):
        raise ValueError(f'{data_path} is not a directory')

    # Traverse through all the directories and read the data
    all_files = glob.glob(os.path.join(data_path, '**/*'), recursive=True)

    # Separate .edf and .csv files
    edf_files = [file for file in all_files if file.endswith('.edf')]
    csv_files = [file for file in all_files if file.endswith('.csv')]

    return edf_files, csv_files

edf_files, csv_files = read_dirs('../dataset/FPE/')    

In [30]:
edf_files [1], csv_files[1]

('../dataset/FPE/S1/p1/fpe_pos1_001_S1_rep0_BT.edf',
 '../dataset/FPE/S1/p1/fpe_pos1_001_S1_rep0_BT.csv')

In [1]:
import sys
sys.path.append('../')
from config import cfg
from data import make_dataset
print(cfg)
cfg.DEBUG = False
cfg.DATA.PATH = '../dataset/FPE/S1/p3'
cfg.DATA.EMG.SAMPLING_RATE = 250
cfg.DATA.SEGMENT_LENGTH = 100
cfg.DATA.STRIDE = 10


dataset = make_dataset(cfg)

DATA:
  EMG:
    BUFF_LEN: 0
    FEATURE_EXTRACTOR: RMS
    HIGH_FREQ: 400
    LOW_FREQ: 10
    NORMALIZATION: max
    NOTCH_FREQ: 50
    NUM_CHANNELS: 16
    Q: 30
    SAMPLING_RATE: 150
    WINDOW_SIZE: 100
    WINDOW_STRIDE: 50
  FILTER_DATA: True
  LABEL_PATH: finger_pose_estimation/dataset/label_2023-10-02_15-24-12_YH_lab_R.csv
  MANUS:
    KEY_POINTS: []
    NUM_JOINTS: 20
    SAMPLING_RATE: 250
  PATH: finger_pose_estimation/dataset/data_2023-10-02 14-59-55-627.edf
  SEGMENT_LENGTH: 150
  STRIDE: 1
  VIDEO:
    NUM_CHANNELS: 3
    SAMPLING_RATE: 30
DEBUG: True
MODEL:
  NAME: transformer
SOLVER:
  BATCH_SIZE: 32
  CHECKPOINT_PERIOD: 10
  DEVICE: cuda
  LOG_DIR: ./outputs
  LR: 0.001
  METRIC: mse
  MOMENTUM: 0.9
  NUM_EPOCHS: 100
  NUM_WORKERS: 4
  OPTIMIZER: Adam
  PATIENCE: 5
  PIN_MEMORY: True
  PRETRAINED_PATH: model.pth
  PRINT_FREQ: 10
  SEED: 42
  WEIGHT_DECAY: 0.0001
VISUALIZE:
  PORT: 9000
Reading data from ../dataset/FPE/S1/p3/fpe_pos3_001_S1_rep0_BT.edf and ../dataset/

ValueError: not enough values to unpack (expected 3, got 2)